In [1]:
import torch

# Perform a CUDA device availability check
def get_cuda_device():
    if torch.cuda.is_available():
        return "cuda:0"
    else:
        return "cpu"

print("Using device:", get_cuda_device())

Using device: cpu


In [2]:
import requests
import json

prompt = "Kolik kalorií má jablko?"

response = requests.post(
    "http://localhost:11434/api/generate",
    json={"model": "mistral", "prompt": prompt},
    stream=True
)

# sběr výstupu
full_response = ""
for line in response.iter_lines():
    if line:
        data = json.loads(line)
        full_response += data.get("response", "")

print(full_response)


Řekněme, aby bylo jednoduché: 62 kalorii je průměrný počet kalorií v jednom jableku. Množství kalorií v jednotlivých jablcích může se lišit podle velikosti a druhu jablka, takže je třeba mít na paměti, že tento počet představuje pouze průměr.


In [3]:
import asyncio
import aiohttp
import json

async def generate_response():
    url = "http://localhost:11434/api/generate"
    payload = {"model": "mistral", "prompt": "Kolik kalorií má jablko?"}

    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=payload) as response:
            full_response = ""
            async for line in response.content:
                if line:
                    try:
                        data = json.loads(line.decode("utf-8").strip())
                        full_response += data.get("response", "")
                    except json.JSONDecodeError:
                        continue  # někdy může přijít neúplný JSON
            print(full_response)

await generate_response()

 Jablko má asi 52 kilokalorii. Tato hodnota se může lišit v závislosti na velikosti, zralosti a typu ovoce.


In [4]:
import requests
import json
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

class OllamaLLM(LLM):
    model: str = "mistral"
    api_url: str = "http://localhost:11434/api/generate"
    system_instructions: str = """
    You are an agent that must strictly follow the output format below.
    Your responses will be run sequentially by a program that will interpret your output.
    ONLY respond in one of these formats per response, NEVER both in the same response.

    Option 1 (for tool use):
    Thought: <your thought>
    Action: <tool name>
    Action Input: <input>

    Option 2 (for final answer):
    Final Answer: <your answer>

    IMPORTANT: Do NOT output both an Action and a Final Answer in the same response.
    If you use a tool, do NOT provide a Final Answer until you have the tool's result.
    """

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        full_prompt = self.system_instructions + "\n" + prompt
        response = requests.post(
            self.api_url,
            json={"model": self.model, "prompt": full_prompt},
            stream=True
        )
        output = ""
        for line in response.iter_lines():
            if line:
                data = json.loads(line)
                output += data.get("response", "")
        return output

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": self.model}

    @property
    def _llm_type(self) -> str:
        return "ollama"

from langchain.tools import BaseTool
import yfinance as yf

class StockPriceTool(BaseTool):
    name: str = "stock_price"
    description: str = "Use this tool to get current stock price by ticker symbol. Input should be a ticker symbol like 'AAPL' or 'GOOG'."

    def _run(self, ticker: str) -> str:
        try:
            stock = yf.Ticker(ticker)
            price = stock.info.get("regularMarketPrice")
            if price:
                return f"The current price of {ticker} is {price} USD."
            else:
                return f"Price for {ticker} not found."
        except Exception as e:
            return f"Error fetching price for {ticker}: {e}"

    async def _arun(self, ticker: str) -> str:
        # Async verze, pokud bys chtěl
        return self._run(ticker)



In [5]:
# AAPL
StockPriceTool()._run("AAPL")

'The current price of AAPL is 203.92 USD.'

In [6]:
import yfinance as yf
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.llms.base import LLM
from langchain.schema import AgentAction, AgentFinish
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents.agent import AgentOutputParser
from typing import Optional, List, Mapping, Any
import requests
import json
import re

In [7]:
# --- Vlastní parser pro výstup LLM ---
class StrictOutputParser(AgentOutputParser):
    def parse(self, text: str) -> AgentAction | AgentFinish:
        text = text.strip()
        if text.startswith("Final Answer:"):
            return AgentFinish(
                return_values={"output": text[len("Final Answer:"):].strip()},
                log=text,
            )
        # Tolerantní regex na mezery a velikost písmen
        pattern = re.compile(
            r"Thought:\s*(.*?)\s*^ *Action:\s*(.*?)\s*^ *Action Input:\s*(.*)",
            re.DOTALL | re.MULTILINE | re.IGNORECASE
        )
        match = pattern.search(text)
        if match:
            thought = match.group(1).strip()
            tool = match.group(2).strip()
            tool_input = match.group(3).strip().strip('"')
            return AgentAction(
                tool=tool,
                tool_input=tool_input,
                log=text,
            )
        raise ValueError(f"Cannot parse output: {text}")

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

ollama_llm = OllamaLLM()
tools = [StockPriceTool()]

agent = initialize_agent(
    tools,
    ollama_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1
)


user_input = "Jaká je cena akcií společnosti apple?"
result = agent.invoke(user_input)
print("Agent:", result)




ValidationError: 1 validation error for LLMChain
input_variables
  Extra inputs are not permitted [type=extra_forbidden, input_value=['input', 'tools', 'agent_scratchpad'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    result = agent.run(user_input)
    print("Agent:", result)